In [1]:
import os
import time
import argparse
import datetime

import torch
import torch.nn as nn
import torch.nn.utils as utils
import torchvision.utils as vutils    
from tensorboardX import SummaryWriter

from DD_model import Model
from DD_loss import ssim
from DD_data import getTrainingTestingData
from DD_utils import AverageMeter, DepthNorm, colorize

In [2]:
# import os
# import time
# import torch
# import sys
# import torch.optim as optim
# from torchvision import transforms as T
# %matplotlib notebook

In [3]:
# from models import SpecialFuseNetModel
# from DenseDepth_model import Encoder, Decoder
# from data_manager import rgbd_gradients_dataset, rgbd_gradients_dataloader
# from train import FuseNetTrainer
# from plot import post_epoch_plot

In [4]:
load_weights_from_file = True
# load_weights_from_file = False

CWD             = os.getcwd()
DATASET_DIR     = os.path.join(CWD,'data/nyuv2')
print(DATASET_DIR)

C:\Users\tomav\Documents\GitHub\cs236781-project\data/nyuv2


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
def LogProgress(model, writer, test_loader, epoch):
    model.eval()
    sequential = test_loader
    sample_batched = next(iter(sequential))
    image = torch.autograd.Variable(sample_batched['image'].cuda())
    depth = torch.autograd.Variable(sample_batched['depth'].cuda(non_blocking=True))
    if epoch == 0: writer.add_image('Train.1.Image', vutils.make_grid(image.data, nrow=6, normalize=True), epoch)
    if epoch == 0: writer.add_image('Train.2.Depth', colorize(vutils.make_grid(depth.data, nrow=6, normalize=False)), epoch)
    output = DepthNorm( model(image) )
    writer.add_image('Train.3.Ours', colorize(vutils.make_grid(output.data, nrow=6, normalize=False)), epoch)
    writer.add_image('Train.3.Diff', colorize(vutils.make_grid(torch.abs(output-depth).data, nrow=6, normalize=False)), epoch)
    del image
    del depth
    del output

# Arguments
parser = argparse.ArgumentParser(description='High Quality Monocular Depth Estimation via Transfer Learning')
parser.add_argument('--epochs', default=10, type=int, help='number of total epochs to run')
parser.add_argument('--lr', '--learning-rate', default=0.0001, type=float, help='initial learning rate')
parser.add_argument('--bs', default=4, type=int, help='batch size')
# args = parser.parse_args()
args, unknown = parser.parse_known_args()

# Create model
model = Model().cuda()
print('Model created.')

# Load model from file
if load_weights_from_file:
    model = torch.load('checkpoints_rgb_to_depth/model_rgb_to_depth.pt')
    print('Model loaded from file.')

# Training parameters
optimizer = torch.optim.Adam( model.parameters(), args.lr )
batch_size = args.bs
batch_size = 2 # LARGEST THAT FITS MEMORY # TOM
prefix = 'densenet_' + str(batch_size)

# Load data
train_loader, test_loader = getTrainingTestingData(batch_size=batch_size)

# Logging
writer = SummaryWriter(comment='{}-lr{}-e{}-bs{}'.format(prefix, args.lr, args.epochs, args.bs), flush_secs=30)

# Loss
l1_criterion = nn.L1Loss()

# Start training...
for epoch in range(args.epochs):
    batch_time = AverageMeter()
    losses = AverageMeter()
    N = len(train_loader)

    # Switch to train mode
    model.train()

    end = time.time()

    for i, sample_batched in enumerate(train_loader):
        optimizer.zero_grad()

        # Prepare sample and target
        image = torch.autograd.Variable(sample_batched['image'].cuda())
        depth = torch.autograd.Variable(sample_batched['depth'].cuda(non_blocking=True))

        # Normalize depth
        depth_n = DepthNorm( depth )

        # Predict
        output = model(image)

        # Compute the loss
        l_depth = l1_criterion(output, depth_n)
        l_ssim = torch.clamp((1 - ssim(output, depth_n, val_range = 1000.0 / 10.0)) * 0.5, 0, 1)

        loss = (1.0 * l_ssim) + (0.1 * l_depth)

        # Update step
        losses.update(loss.data.item(), image.size(0))
        loss.backward()
        optimizer.step()

        # Measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        eta = str(datetime.timedelta(seconds=int(batch_time.val*(N - i))))

        # Log progress
        niter = epoch*N+i
        if i % 5 == 0:
            # Print to console
            print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.sum:.3f})\t'
            'ETA {eta}\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})'
            .format(epoch, i, N, batch_time=batch_time, loss=losses, eta=eta))

            # Log to tensorboard
            writer.add_scalar('Train/Loss', losses.val, niter)

        if i % 300 == 0:
            LogProgress(model, writer, test_loader, niter)

    # Record epoch's intermediate results
    LogProgress(model, writer, test_loader, niter)
    writer.add_scalar('Train/Loss.avg', losses.avg, epoch)
    
    torch.save(model, "model_rgb_to_depth.pt")

Model created.
Model loaded from file.
Loading dataset zip file...Loaded (50688).
Epoch: [0][0/25344]	Time 2.299 (2.299)	ETA 16:11:01	Loss 0.3793 (0.3793)
Epoch: [0][5/25344]	Time 0.641 (5.997)	ETA 4:30:48	Loss 0.0830 (0.1234)
Epoch: [0][10/25344]	Time 0.629 (9.050)	ETA 4:25:31	Loss 0.0272 (0.0882)
Epoch: [0][15/25344]	Time 0.625 (12.026)	ETA 4:23:41	Loss 0.0461 (0.0716)
Epoch: [0][20/25344]	Time 0.645 (15.096)	ETA 4:32:26	Loss 0.0261 (0.0628)
Epoch: [0][25/25344]	Time 0.597 (18.148)	ETA 4:12:07	Loss 0.0677 (0.0617)
Epoch: [0][30/25344]	Time 0.551 (21.056)	ETA 3:52:25	Loss 0.0350 (0.0588)
Epoch: [0][35/25344]	Time 0.645 (24.063)	ETA 4:32:09	Loss 0.0215 (0.0548)
Epoch: [0][40/25344]	Time 0.536 (26.904)	ETA 3:45:59	Loss 0.0290 (0.0526)
Epoch: [0][45/25344]	Time 0.623 (30.120)	ETA 4:22:49	Loss 0.0391 (0.0508)
Epoch: [0][50/25344]	Time 0.585 (33.175)	ETA 4:06:40	Loss 0.0375 (0.0498)
Epoch: [0][55/25344]	Time 0.576 (36.136)	ETA 4:02:53	Loss 0.0234 (0.0482)
Epoch: [0][60/25344]	Time 0.666 (3

Epoch: [0][540/25344]	Time 0.597 (330.684)	ETA 4:06:40	Loss 0.0647 (0.0353)
Epoch: [0][545/25344]	Time 0.581 (333.755)	ETA 4:00:18	Loss 0.0459 (0.0352)
Epoch: [0][550/25344]	Time 0.605 (336.715)	ETA 4:09:50	Loss 0.0343 (0.0353)
Epoch: [0][555/25344]	Time 0.659 (339.708)	ETA 4:32:25	Loss 0.0279 (0.0353)
Epoch: [0][560/25344]	Time 0.552 (342.688)	ETA 3:48:08	Loss 0.0410 (0.0354)
Epoch: [0][565/25344]	Time 0.562 (345.739)	ETA 3:52:09	Loss 0.0394 (0.0354)
Epoch: [0][570/25344]	Time 0.560 (348.799)	ETA 3:51:17	Loss 0.0364 (0.0354)
Epoch: [0][575/25344]	Time 0.600 (351.745)	ETA 4:07:35	Loss 0.0369 (0.0353)
Epoch: [0][580/25344]	Time 0.668 (354.885)	ETA 4:35:43	Loss 0.0400 (0.0353)
Epoch: [0][585/25344]	Time 0.567 (357.998)	ETA 3:54:08	Loss 0.0314 (0.0352)
Epoch: [0][590/25344]	Time 0.588 (361.010)	ETA 4:02:38	Loss 0.0452 (0.0352)
Epoch: [0][595/25344]	Time 1.072 (364.592)	ETA 7:22:17	Loss 0.0345 (0.0352)
Epoch: [0][600/25344]	Time 0.593 (368.129)	ETA 4:04:22	Loss 0.0189 (0.0352)
Epoch: [0][6

Epoch: [0][1080/25344]	Time 0.562 (657.539)	ETA 3:47:23	Loss 0.0332 (0.0355)
Epoch: [0][1085/25344]	Time 0.573 (660.603)	ETA 3:51:47	Loss 0.0337 (0.0355)
Epoch: [0][1090/25344]	Time 0.572 (663.711)	ETA 3:51:21	Loss 0.0409 (0.0354)
Epoch: [0][1095/25344]	Time 0.679 (666.821)	ETA 4:34:33	Loss 0.0332 (0.0354)
Epoch: [0][1100/25344]	Time 0.643 (669.877)	ETA 4:19:44	Loss 0.0742 (0.0355)
Epoch: [0][1105/25344]	Time 0.645 (672.997)	ETA 4:20:33	Loss 0.0301 (0.0354)
Epoch: [0][1110/25344]	Time 0.616 (676.250)	ETA 4:08:43	Loss 0.0211 (0.0355)
Epoch: [0][1115/25344]	Time 0.651 (679.435)	ETA 4:23:04	Loss 0.0228 (0.0354)
Epoch: [0][1120/25344]	Time 0.651 (682.468)	ETA 4:23:01	Loss 0.0311 (0.0354)
Epoch: [0][1125/25344]	Time 0.563 (685.471)	ETA 3:47:24	Loss 0.0282 (0.0354)
Epoch: [0][1130/25344]	Time 0.581 (688.426)	ETA 3:54:38	Loss 0.0553 (0.0354)
Epoch: [0][1135/25344]	Time 0.623 (691.540)	ETA 4:11:21	Loss 0.0219 (0.0354)
Epoch: [0][1140/25344]	Time 0.606 (694.651)	ETA 4:04:21	Loss 0.0308 (0.0354)

Epoch: [0][1615/25344]	Time 0.534 (985.120)	ETA 3:31:11	Loss 0.0602 (0.0348)
Epoch: [0][1620/25344]	Time 0.590 (987.863)	ETA 3:53:15	Loss 0.0413 (0.0348)
Epoch: [0][1625/25344]	Time 0.654 (990.898)	ETA 4:18:25	Loss 0.0263 (0.0348)
Epoch: [0][1630/25344]	Time 0.615 (993.864)	ETA 4:02:52	Loss 0.0154 (0.0348)
Epoch: [0][1635/25344]	Time 0.620 (996.968)	ETA 4:05:03	Loss 0.0329 (0.0348)
Epoch: [0][1640/25344]	Time 0.621 (999.904)	ETA 4:05:13	Loss 0.0189 (0.0347)
Epoch: [0][1645/25344]	Time 0.610 (1002.806)	ETA 4:00:58	Loss 0.0361 (0.0347)
Epoch: [0][1650/25344]	Time 0.641 (1005.753)	ETA 4:13:03	Loss 0.0272 (0.0347)
Epoch: [0][1655/25344]	Time 0.573 (1008.827)	ETA 3:46:07	Loss 0.0607 (0.0348)
Epoch: [0][1660/25344]	Time 0.614 (1011.850)	ETA 4:02:15	Loss 0.0389 (0.0347)
Epoch: [0][1665/25344]	Time 0.591 (1014.889)	ETA 3:53:09	Loss 0.0579 (0.0347)
Epoch: [0][1670/25344]	Time 0.548 (1017.763)	ETA 3:36:16	Loss 0.0270 (0.0347)
Epoch: [0][1675/25344]	Time 0.553 (1020.767)	ETA 3:38:18	Loss 0.0262 (

Epoch: [0][2145/25344]	Time 0.642 (1300.715)	ETA 4:08:19	Loss 0.0397 (0.0346)
Epoch: [0][2150/25344]	Time 0.598 (1303.776)	ETA 3:51:09	Loss 0.0169 (0.0346)
Epoch: [0][2155/25344]	Time 0.537 (1306.686)	ETA 3:27:38	Loss 0.0282 (0.0346)
Epoch: [0][2160/25344]	Time 0.620 (1309.724)	ETA 3:59:38	Loss 0.0158 (0.0346)
Epoch: [0][2165/25344]	Time 0.633 (1312.759)	ETA 4:04:35	Loss 0.0426 (0.0346)
Epoch: [0][2170/25344]	Time 0.637 (1315.774)	ETA 4:05:59	Loss 0.0318 (0.0345)
Epoch: [0][2175/25344]	Time 0.596 (1318.778)	ETA 3:50:10	Loss 0.0219 (0.0345)
Epoch: [0][2180/25344]	Time 0.616 (1321.739)	ETA 3:57:42	Loss 0.0323 (0.0345)
Epoch: [0][2185/25344]	Time 0.546 (1324.726)	ETA 3:30:40	Loss 0.0266 (0.0345)
Epoch: [0][2190/25344]	Time 0.553 (1327.731)	ETA 3:33:32	Loss 0.0354 (0.0345)
Epoch: [0][2195/25344]	Time 0.617 (1330.753)	ETA 3:57:55	Loss 0.0371 (0.0345)
Epoch: [0][2200/25344]	Time 0.535 (1333.440)	ETA 3:26:22	Loss 0.0314 (0.0345)
Epoch: [0][2205/25344]	Time 0.527 (1336.274)	ETA 3:23:25	Loss 0.

...

Epoch: [0][19425/25344]	Time 2.085 (12061.421)	ETA 3:25:41	Loss 0.0441 (0.0330)
Epoch: [0][19430/25344]	Time 2.269 (12072.750)	ETA 3:43:37	Loss 0.0274 (0.0330)
Epoch: [0][19435/25344]	Time 2.053 (12083.591)	ETA 3:22:13	Loss 0.0221 (0.0330)
Epoch: [0][19440/25344]	Time 2.034 (12093.883)	ETA 3:20:08	Loss 0.0367 (0.0330)
Epoch: [0][19445/25344]	Time 2.214 (12104.461)	ETA 3:37:39	Loss 0.0242 (0.0330)
Epoch: [0][19450/25344]	Time 2.105 (12115.166)	ETA 3:26:49	Loss 0.0391 (0.0330)
Epoch: [0][19455/25344]	Time 2.128 (12125.551)	ETA 3:28:51	Loss 0.0185 (0.0330)
Epoch: [0][19460/25344]	Time 2.060 (12136.037)	ETA 3:21:59	Loss 0.0325 (0.0330)
Epoch: [0][19465/25344]	Time 2.018 (12146.183)	ETA 3:17:45	Loss 0.0272 (0.0330)
Epoch: [0][19470/25344]	Time 2.189 (12156.856)	ETA 3:34:19	Loss 0.0272 (0.0330)
Epoch: [0][19475/25344]	Time 2.189 (12167.587)	ETA 3:34:09	Loss 0.0382 (0.0330)
Epoch: [0][19480/25344]	Time 2.178 (12178.074)	ETA 3:32:52	Loss 0.0294 (0.0330)
Epoch: [0][19485/25344]	Time 2.095 (1218

Epoch: [0][19940/25344]	Time 1.133 (12798.929)	ETA 1:42:02	Loss 0.0324 (0.0329)
Epoch: [0][19945/25344]	Time 1.249 (12804.682)	ETA 1:52:21	Loss 0.0343 (0.0329)
Epoch: [0][19950/25344]	Time 1.160 (12810.387)	ETA 1:44:16	Loss 0.0469 (0.0329)
Epoch: [0][19955/25344]	Time 1.130 (12816.333)	ETA 1:41:29	Loss 0.0417 (0.0329)
Epoch: [0][19960/25344]	Time 1.131 (12821.980)	ETA 1:41:28	Loss 0.0231 (0.0329)
Epoch: [0][19965/25344]	Time 1.120 (12827.644)	ETA 1:40:22	Loss 0.0285 (0.0329)
Epoch: [0][19970/25344]	Time 1.127 (12833.331)	ETA 1:40:55	Loss 0.0388 (0.0329)
Epoch: [0][19975/25344]	Time 1.134 (12839.197)	ETA 1:41:27	Loss 0.0208 (0.0329)
Epoch: [0][19980/25344]	Time 1.142 (12844.841)	ETA 1:42:03	Loss 0.0159 (0.0329)
Epoch: [0][19985/25344]	Time 1.215 (12851.055)	ETA 1:48:30	Loss 0.0369 (0.0329)
Epoch: [0][19990/25344]	Time 1.255 (12857.403)	ETA 1:51:59	Loss 0.0355 (0.0329)
Epoch: [0][19995/25344]	Time 1.270 (12863.509)	ETA 1:53:13	Loss 0.0430 (0.0329)
Epoch: [0][20000/25344]	Time 1.179 (1286

Epoch: [0][20455/25344]	Time 1.152 (13425.462)	ETA 1:33:52	Loss 0.0269 (0.0329)
Epoch: [0][20460/25344]	Time 1.177 (13431.246)	ETA 1:35:46	Loss 0.0389 (0.0329)
Epoch: [0][20465/25344]	Time 1.175 (13437.072)	ETA 1:35:31	Loss 0.0222 (0.0329)
Epoch: [0][20470/25344]	Time 1.161 (13442.831)	ETA 1:34:20	Loss 0.0228 (0.0329)
Epoch: [0][20475/25344]	Time 1.224 (13448.781)	ETA 1:39:18	Loss 0.0263 (0.0329)
Epoch: [0][20480/25344]	Time 1.160 (13454.658)	ETA 1:34:04	Loss 0.0299 (0.0329)
Epoch: [0][20485/25344]	Time 1.250 (13460.587)	ETA 1:41:15	Loss 0.0416 (0.0329)
Epoch: [0][20490/25344]	Time 1.256 (13466.832)	ETA 1:41:34	Loss 0.0277 (0.0329)
Epoch: [0][20495/25344]	Time 1.161 (13472.989)	ETA 1:33:51	Loss 0.0246 (0.0329)
Epoch: [0][20500/25344]	Time 1.179 (13478.947)	ETA 1:35:09	Loss 0.0345 (0.0329)
Epoch: [0][20505/25344]	Time 1.172 (13484.856)	ETA 1:34:28	Loss 0.0208 (0.0329)
Epoch: [0][20510/25344]	Time 1.157 (13490.691)	ETA 1:33:11	Loss 0.0258 (0.0329)
Epoch: [0][20515/25344]	Time 1.262 (1349

Epoch: [0][20970/25344]	Time 1.207 (14078.190)	ETA 1:27:59	Loss 0.0236 (0.0329)
Epoch: [0][20975/25344]	Time 1.213 (14084.225)	ETA 1:28:21	Loss 0.0213 (0.0329)
Epoch: [0][20980/25344]	Time 1.210 (14090.299)	ETA 1:28:01	Loss 0.0360 (0.0329)
Epoch: [0][20985/25344]	Time 1.190 (14096.289)	ETA 1:26:29	Loss 0.0192 (0.0329)
Epoch: [0][20990/25344]	Time 1.212 (14102.294)	ETA 1:27:56	Loss 0.0301 (0.0329)
Epoch: [0][20995/25344]	Time 1.188 (14108.301)	ETA 1:26:05	Loss 0.0596 (0.0329)
Epoch: [0][21000/25344]	Time 1.240 (14114.834)	ETA 1:29:45	Loss 0.0201 (0.0329)
Epoch: [0][21005/25344]	Time 1.244 (14122.497)	ETA 1:29:59	Loss 0.1241 (0.0329)
Epoch: [0][21010/25344]	Time 1.207 (14128.602)	ETA 1:27:09	Loss 0.0219 (0.0329)
Epoch: [0][21015/25344]	Time 1.439 (14135.134)	ETA 1:43:50	Loss 0.0272 (0.0329)
Epoch: [0][21020/25344]	Time 1.378 (14142.195)	ETA 1:39:18	Loss 0.0279 (0.0329)
Epoch: [0][21025/25344]	Time 1.357 (14149.124)	ETA 1:37:38	Loss 0.0191 (0.0329)
Epoch: [0][21030/25344]	Time 1.303 (1415

Epoch: [0][21485/25344]	Time 2.533 (14906.543)	ETA 2:42:55	Loss 0.0247 (0.0329)
Epoch: [0][21490/25344]	Time 2.516 (14919.232)	ETA 2:41:36	Loss 0.0451 (0.0329)
Epoch: [0][21495/25344]	Time 1.685 (14928.525)	ETA 1:48:07	Loss 0.0433 (0.0329)
Epoch: [0][21500/25344]	Time 1.856 (14937.344)	ETA 1:58:56	Loss 0.0401 (0.0329)
Epoch: [0][21505/25344]	Time 1.726 (14946.210)	ETA 1:50:26	Loss 0.0273 (0.0329)
Epoch: [0][21510/25344]	Time 1.755 (14954.898)	ETA 1:52:08	Loss 0.0252 (0.0329)
Epoch: [0][21515/25344]	Time 1.724 (14963.618)	ETA 1:49:59	Loss 0.0258 (0.0329)
Epoch: [0][21520/25344]	Time 1.787 (14972.499)	ETA 1:53:52	Loss 0.0198 (0.0329)
Epoch: [0][21525/25344]	Time 1.782 (14981.245)	ETA 1:53:25	Loss 0.0388 (0.0329)
Epoch: [0][21530/25344]	Time 1.869 (14990.242)	ETA 1:58:48	Loss 0.0211 (0.0329)
Epoch: [0][21535/25344]	Time 2.116 (14999.926)	ETA 2:14:21	Loss 0.0221 (0.0329)
Epoch: [0][21540/25344]	Time 2.028 (15009.861)	ETA 2:08:35	Loss 0.0328 (0.0329)
Epoch: [0][21545/25344]	Time 2.061 (1501